In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
# read files
with open('S21-gene-train.txt') as f:
    lines = f.read()

In [3]:
lines=lines.split("\n")
sentences=[]
temp_list=[]
for line in lines:
    if line !='':
        temp_list.append(line)
    else:
        sentences.append(temp_list)
        temp_list=[]
        

In [4]:
train_sentences=sentences[:int(len(sentences)*0.8)]
dev_sentences=sentences[int(len(sentences)*0.8):]

In [5]:
train_data=[]
for s in train_sentences:
    for word in s:
        train_data.append(word.split("\t"))

In [6]:
df=pd.DataFrame(train_data,columns=["seq","word","tag"])


In [7]:
# replace rare words(words only appear  once) with "<unk>"
rare_words=[key for key,val in Counter(df["word"].to_list()).items() if val==1]
replace_map = dict(zip(rare_words, ["<unk>" for _ in range(len(rare_words))]))
df["word"] = df["word"].map(lambda x: replace_map.get(x,x))

In [8]:
def compute_start_prob(df):
    #component sequence I, O, B
    IOB_prob_dict={'I':None,'O':None,'B':None}
    df_head=df[df["seq"]=="1"]
    head_size=df_head.shape[0]
    for tag in ["I","O","B"]:
        try:
            value=df_head.groupby(['tag']).size()[tag]
            IOB_prob_dict[tag]=value/head_size
        except:
            IOB_prob_dict[tag]=0
    df_head.groupby(['tag']).size()
    print("start prob:")
    print(IOB_prob_dict)
    return IOB_prob_dict

In [9]:
def compute_transi_prob(sentences):
    df=pd.DataFrame(0,index=["I","O","B"],columns=["I","O","B"])
    for s in sentences:
        tags=[x.split("\t")[2] for x in s]
        for i in range(1,len(tags)):
            pre=tags[i-1]
            cur=tags[i]
            df.at[pre, cur]=df.at[pre, cur]+1
            
    IOB_sum=df.sum(axis=1)    
    df=df.divide(IOB_sum,axis=0)
    print(df)
    return df.transpose().to_dict()
            
    

In [10]:
def compute_emmision_prob(df):
    I_dict=Counter(df.groupby(["tag"]).get_group("I")["word"].to_list())
    O_dict=Counter(df.groupby(["tag"]).get_group("O")["word"].to_list())
    B_dict=Counter(df.groupby(["tag"]).get_group("B")["word"].to_list())
    IOB_dict={"I":I_dict,"O":O_dict,"B":B_dict}
    word_set=list(set.union(set(I_dict.keys()),set(O_dict.keys()),set(B_dict.keys())))
    # Lapace smoothing
    z=10
    emmision_matrix={"I":dict.fromkeys(word_set, z/sum(IOB_dict["I"].values())),"O":dict.fromkeys(word_set, z/sum(IOB_dict["O"].values())),"B":dict.fromkeys(word_set,z/sum(IOB_dict["B"].values()))}
    for tag in ["I","O","B"]:
        tag_sum=sum(IOB_dict[tag].values())
        for key, values in IOB_dict[tag].items():
            emmision_matrix[tag][key]=(IOB_dict[tag][key]+z)/(tag_sum+z*2)
    return emmision_matrix


In [11]:
observations = [x.split("\t")[1] for x in sentences[0]]
states = ("I", "O", "B")
start_p = compute_start_prob(df)#{"I": 0, "O": 0.9457089, "B":0.0542911}
trans_p = compute_transi_prob(sentences)
emit_p = compute_emmision_prob(df)


start prob:
{'I': 0, 'O': 0.9516129032258065, 'B': 0.04838709677419355}
          I         O        B
I  0.606018  0.393982  0.00000
O  0.000000  0.952050  0.04795
B  0.579080  0.420920  0.00000


In [12]:
# code reference: https://en.wikipedia.org/wiki/Viterbi_algorithm
def viterbi_algorithm(observations, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        if observations[0] not in emit_p[st].keys():
                observations[0]="<unk>"
        V[0][st] = {"prob": np.log(start_p[st]) + np.log(emit_p[st][observations[0]]), "prev": None}
   
    for t in range(1, len(observations)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] + np.log(trans_p[states[0]][st])
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] + np.log(trans_p[prev_st][st])
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
            # handle unkown words
            if observations[t] not in emit_p[st].keys():
                observations[t]="<unk>"
            max_prob = max_tr_prob + np.log(emit_p[st][observations[t]])
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
    for line in dptable(V):
        print(line)
    opt = []
    max_prob = float('-inf')
    best_st = None
 
    for st, data in V[-1].items():
        print(st,data)
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st
 
 
    for t in range(len(V) - 2, -1, -1):
        try:
            opt.insert(0, V[t + 1][previous]["prev"])
        except:
            for v in V:
                for st, data in v.items():
                    print(st,data)
                print()
        previous = V[t + 1][previous]["prev"]
 
    print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)
    return opt
def dptable(V):
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%lf" % v[state]["prob"]) for v in V)

In [13]:
result_list=[]
for sent in dev_sentences:
    observations=[x.split("\t")[1] for x in sent]
    print(len(observations))
    result=viterbi_algorithm(observations, states, start_p, trans_p, emit_p)
    result_list.append([s[:-1]+r for s,r in zip(sent,result)])
    

28
       0       1       2       3       4       5       6       7       8       9      10      11      12      13      14      15      16      17      18      19      20      21      22      23      24      25      26      27
I: -inf -17.607 -25.626 -33.051 -41.070 -49.090 -55.651 -59.328 -62.764 -70.783 -76.289 -82.341 -90.360 -98.380 -105.41 -111.45 -117.00 -125.02 -125.16 -127.59 -135.61 -141.11 -147.17 -155.19 -163.20 -171.22 -179.15 -184.50
O: -9.2099 -15.317 -25.409 -32.476 -37.448 -46.470 -51.147 -54.377 -58.180 -64.138 -73.770 -82.792 -87.469 -96.955 -104.26 -107.69 -111.30 -115.97 -119.20 -123.01 -128.96 -138.60 -147.62 -155.42 -164.91 -171.77 -181.25 -183.19
B: -10.129 -19.348 -25.455 -35.547 -42.615 -47.587 -56.608 -55.728 -64.516 -68.318 -74.276 -83.909 -92.930 -97.607 -107.09 -114.40 -117.83 -121.43 -120.55 -129.34 -133.14 -139.10 -148.73 -157.76 -165.56 -174.95 -181.90 -191.39
I {'prob': -184.50027932888975, 'prev': 'I'}
O {'prob': -183.19524266556257, 'prev': 'I'}
B {'

/var/folders/yw/r95kjnwn1675n9x1d3jjsgrh0000gn/T/ipykernel_10130/3847970862.py:7: RuntimeWarning: divide by zero encountered in log
  V[0][st] = {"prob": np.log(start_p[st]) + np.log(emit_p[st][observations[0]]), "prev": None}
/var/folders/yw/r95kjnwn1675n9x1d3jjsgrh0000gn/T/ipykernel_10130/3847970862.py:15: RuntimeWarning: divide by zero encountered in log
  tr_prob = V[t - 1][prev_st]["prob"] + np.log(trans_p[prev_st][st])
/var/folders/yw/r95kjnwn1675n9x1d3jjsgrh0000gn/T/ipykernel_10130/3847970862.py:12: RuntimeWarning: divide by zero encountered in log
  max_tr_prob = V[t - 1][states[0]]["prob"] + np.log(trans_p[states[0]][st])


21
       0       1       2       3       4       5       6       7       8       9      10      11      12      13      14      15      16      17      18      19      20
I: -inf -13.851 -13.055 -18.374 -24.420 -31.748 -39.126 -41.303 -49.322 -57.160 -65.179 -68.970 -74.317 -80.309 -85.008 -90.327 -94.760 -101.61 -107.35 -114.18 -119.52
O: -6.1904 -9.4211 -13.164 -19.276 -22.679 -25.827 -36.101 -42.159 -47.234 -54.944 -61.527 -67.243 -70.402 -76.362 -79.970 -86.082 -89.241 -96.159 -105.33 -114.17 -117.33
B: -10.129 -10.771 -19.378 -22.562 -29.415 -32.818 -35.012 -46.240 -52.297 -57.373 -65.082 -71.261 -77.381 -79.410 -86.501 -89.368 -96.221 -99.286 -106.29 -115.47 -124.31
I {'prob': -119.52700812544713, 'prev': 'I'}
O {'prob': -117.33261026512369, 'prev': 'O'}
B {'prob': -124.31215409653616, 'prev': 'O'}
The steps of states are O B I I O O B O O O O O O O O O O O O O O with highest probability of -117.33261026512369
72
       0       1       2       3       4       5       6       7  

In [14]:
def write_result(result_list):
    textfile = open("result_file.txt", "w")
    for sentences in result_list:
        for element in sentences:
            textfile.write(element + "\n")
        textfile.write("\n")
    textfile.close()

In [15]:
write_result(result_list)

In [16]:
def write_gold_ans(result_list):
    textfile = open("golden_ans_file.txt", "w")
    for sentences in result_list:
        for element in sentences:
            textfile.write(element + "\n")
        textfile.write("\n")
    textfile.close()

write_gold_ans(dev_sentences)


In [ ]:
'''
##### Test result  #####
(4508, ' entities in gold standard.')
(4839, ' total entities found.')
(851, ' of which were correct.')
('\t', 'Precision: ', 0.17586278156643934, 'Recall: ', 0.18877551020408162, 'F1-measure: ', 0.18209051032416818)
weitungliao@WEITUNGs-MacBook-Pro NLP Assignment3 % 
'''